In [1]:
import random
import numpy as np
import sklearn
from sklearn.datasets import load_iris

In [2]:
r = [i for i in range(150)]
random.shuffle(r)
iris_data = (load_iris()['data'][r], load_iris()['target'][r])

In [3]:
def vectorized_result(j):
    e = np.zeros((3, 1))
    e[j] = 1.0
    return e

In [4]:
split = int(0.8 * 150)
tr_d = (iris_data[0][:split], iris_data[1][:split])
te_d = (iris_data[0][split:], iris_data[1][split:])

training_inputs  = [np.reshape(x, (4, 1)) for x in tr_d[0]]
training_results = [vectorized_result(y)  for y in tr_d[1]]
training_data    = list(zip(training_inputs, training_results))

test_inputs = [np.reshape(x, (4, 1)) for x in te_d[0]]
test_data   = list(zip(test_inputs, te_d[1]))

In [5]:
num_layers = 0
sizes      = list()
biases     = list()
weights    = list()

In [6]:
def network(sizes_p) :
    global num_layers
    num_layers = len(sizes_p)
    
    global sizes
    sizes      = sizes_p
    
    global biases
    biases     = [np.random.randn(y, 1) for y in sizes[1:]]
    
    global weights
    weights    = [np.random.randn(y, x)/np.sqrt(x) for x, y in list(zip(sizes[:-1], sizes[1:]))]

In [7]:
network([4, 4, 3])

In [8]:
def feedforward(a) :
    global biases
    global weights
    
    for b, w in list(zip(biases, weights)) :
        a = sigmoid(np.dot(w, a)+b)
    return a

In [9]:
def SGD(training_data, epochs, mini_batch_size, eta, test_data = None) :
    
    if test_data :
        n_test = len(test_data)
    
    n = len(training_data)
    
    for j in range(epochs):
        random.shuffle(training_data)
        mini_batches = [training_data[k : k + mini_batch_size] for k in range(0, n, mini_batch_size)]
            
        for mini_batch in mini_batches:
            update_mini_batch(mini_batch, eta)
        if test_data:
            print("Epoch {0}: {1} / {2}".format(j, evaluate(test_data), n_test))
        else:
            print("Epoch {0} complete".format(j))

In [10]:
def update_mini_batch(mini_batch, eta) :
    
    global biases
    global weights
    
    nabla_b = [np.zeros(b.shape) for b in biases]
    nabla_w = [np.zeros(w.shape) for w in weights]
    
    for x, y in mini_batch :
        
        delta_nabla_b, delta_nabla_w = backprop(x, y)
        
        nabla_b = [nb + dnb for nb, dnb in list(zip(nabla_b, delta_nabla_b))]
        nabla_w = [nw + dnw for nw, dnw in list(zip(nabla_w, delta_nabla_w))]
        
    weights = [w - (eta / len(mini_batch)) * nw for w, nw in list(zip(weights, nabla_w))]
    biases  = [b - (eta / len(mini_batch)) * nb for b, nb in list(zip(biases,  nabla_b))]

In [11]:
def backprop(x, y) :
    
    global biases
    global weights
    
    nabla_b = [np.zeros(b.shape) for b in biases]
    nabla_w = [np.zeros(w.shape) for w in weights]
    
        # feedforward
    activation = x
    activations = [x] 
    zs = [] 
    
    for b, w in list(zip(biases, weights)) :
        z = np.dot(w, activation) + b
        zs.append(z)
        activation = sigmoid(z)
        activations.append(activation)
        
        # backward pass
    delta       = cost_derivative(activations[-1], y)
    nabla_b[-1] = delta
    nabla_w[-1] = np.dot(delta, activations[-2].transpose())
    
    for l in range(2, num_layers):
        z = zs[-l]
        sp = sigmoid_prime(z)
        delta = np.dot(weights[-l+1].transpose(), delta) * sp
        nabla_b[-l] = delta
        nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
    
    return (nabla_b, nabla_w)

In [12]:
def evaluate(test_data) :
    
    test_results = [(np.argmax(feedforward(x)), y) for (x, y) in test_data]
    
    return sum(int(x == y) for (x, y) in test_results)

In [13]:
def cost_derivative(output_activations, y) :
    
    return (output_activations-y)

In [14]:
def sigmoid(z):
    
    return (1.0 / (1.0 + np.exp(-z)))

In [15]:
def sigmoid_prime(z):
    
    return (sigmoid(z) * (1 - sigmoid(z)))

In [98]:
SGD(training_data, 7, 15, 0.08, test_data = test_data)

Epoch 0: 22 / 30
Epoch 1: 22 / 30
Epoch 2: 22 / 30
Epoch 3: 22 / 30
Epoch 4: 22 / 30
Epoch 5: 22 / 30
Epoch 6: 29 / 30


In [17]:
type(training_data)

list

In [83]:
weights #all 22

[array([[-2.31571157, -1.25988987, -1.27986348, -0.60790617],
        [-0.60993532, -0.54248185, -1.67735235, -0.18150551],
        [-0.71157045, -0.32667726, -1.30194108,  0.01759045],
        [ 0.8721968 ,  3.01925872, -4.02105602, -1.57803457]]),
 array([[-0.6782173 , -1.10709385, -0.06800146,  7.83825989],
        [-0.19956412,  1.21965647,  0.12423629, -4.45720373],
        [-0.29210751, -1.07488829,  0.07680355, -4.56525082]])]

In [84]:
biases

[array([[-0.36518369],
        [-1.86651202],
        [-1.47424023],
        [-1.35289384]]), array([[-4.13176709],
        [-0.07749954],
        [ 0.04281872]])]

In [96]:
global weights
weights = [np.array([[-2.31571157, -1.25988987, -1.27986348, -0.60790617],
        [-0.60993532, -0.54248185, -1.67735235, -0.18150551],
        [-0.71157045, -0.32667726, -1.30194108,  0.01759045],
        [ 0.8721968 ,  3.01925872, -4.02105602, -1.57803457]]),
 np.array([[-0.6782173 , -1.10709385, -0.06800146,  7.83825989],
        [-0.19956412,  1.21965647,  0.12423629, -4.45720373],
        [-0.29210751, -1.07488829,  0.07680355, -4.56525082]])]



In [97]:
global biases
biases = [np.array([[-0.36518369],
        [-1.86651202],
        [-1.47424023],
        [-1.35289384]]), np.array([[-4.13176709],
        [-0.07749954],
        [ 0.04281872]])]

In [99]:
weights, biases #one 29

([array([[-2.31571157, -1.25988987, -1.27986348, -0.60790617],
         [-0.60994635, -0.54249142, -1.67735179, -0.18150451],
         [-0.71157779, -0.32668262, -1.30194257,  0.01759036],
         [ 0.87198157,  3.01974321, -4.02227775, -1.57858206]]),
  array([[-0.6782173 , -1.10709033, -0.06799001,  7.87227777],
         [-0.19956412,  1.21965563,  0.12423666, -4.47104812],
         [-0.29210751, -1.07489057,  0.07679305, -4.58161608]])],
 [array([[-0.36518369],
         [-1.86651449],
         [-1.47424177],
         [-1.35284137]]), array([[-4.14607664],
         [-0.01612644],
         [-0.01612065]])])

In [100]:
evaluate(test_data)

29